In [1]:
!pip install transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("./emoji_math.csv")

dataset = Dataset.from_pandas(df)


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/deepseek-math-7b-rl"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

In [5]:
def tokenize_function(example):
    prompt = f"Incorrect: {example['input']}\nCorrect: {example['output']}"

    tokenized_output = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)

    return {
        "input_ids": tokenized_output["input_ids"],
        "attention_mask": tokenized_output["attention_mask"],
        "labels": tokenized_output["input_ids"]
    }

tokenized_dataset = dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [6]:
from peft import LoraConfig, get_peft_model, TaskType


lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./emojIQ",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=50,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    evaluation_strategy="no",
    gradient_accumulation_steps=4,
    fp16=True,
    push_to_hub=False,
    remove_unused_columns=False,
    save_total_limit=2,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer.train()

Step,Training Loss
10,1.169800
20,0.215600
30,0.128400
40,0.082900
50,0.060600
60,0.054800
70,0.049600
80,0.048100
90,0.046600
100,0.045500


TrainOutput(global_step=300, training_loss=0.08784539510806401, metrics={'train_runtime': 2342.0825, 'train_samples_per_second': 1.046, 'train_steps_per_second': 0.128, 'total_flos': 4.204484195726131e+16, 'train_loss': 0.08784539510806401, 'epoch': 42.96})

In [18]:
model.save_pretrained("./emojIQ")
tokenizer.save_pretrained("./emojIQ")


('./emojIQ/tokenizer_config.json',
 './emojIQ/special_tokens_map.json',
 './emojIQ/tokenizer.json')

In [19]:
!zip -r emojIQ_deepseekmath-r1.zip ./emojIQ

  adding: emojIQ/ (stored 0%)
  adding: emojIQ/adapter_model.safetensors (deflated 7%)
  adding: emojIQ/special_tokens_map.json (deflated 63%)
  adding: emojIQ/checkpoint-294/ (stored 0%)
  adding: emojIQ/checkpoint-294/adapter_model.safetensors (deflated 7%)
  adding: emojIQ/checkpoint-294/README.md (deflated 66%)
  adding: emojIQ/checkpoint-294/optimizer.pt (deflated 8%)
  adding: emojIQ/checkpoint-294/adapter_config.json (deflated 54%)
  adding: emojIQ/checkpoint-294/scheduler.pt (deflated 55%)
  adding: emojIQ/checkpoint-294/rng_state.pth (deflated 25%)
  adding: emojIQ/checkpoint-294/trainer_state.json (deflated 78%)
  adding: emojIQ/checkpoint-294/training_args.bin (deflated 52%)
  adding: emojIQ/README.md (deflated 66%)
  adding: emojIQ/tokenizer_config.json (deflated 65%)
  adding: emojIQ/checkpoint-300/ (stored 0%)
  adding: emojIQ/checkpoint-300/adapter_model.safetensors (deflated 7%)
  adding: emojIQ/checkpoint-300/README.md (deflated 66%)
  adding: emojIQ/checkpoint-300/opt

In [20]:
from IPython.display import FileLink
FileLink(r'emojIQ_deepseekmath-r1.zip')

/content/emojIQ_deepseekmath-r1.zip

In [21]:
def generate_result(incorrect_math):
    input_text = f"Incorrect: {incorrect_math}\nCorrect:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_result("🍔 + 🍔 + 🍔 = 15 → 🍔 = ❓"))
print(generate_result("🚜 + 🚜 + 🚜 + 🚜 = 40 → 🚜 = ❓"))
print(generate_result("🐧 + 🐧 = 18 → 🐧 = ❓"))
print(generate_result("🎈 + 🎈 + 🎈 + 🎈 + 🎈 = 25 → 🎈 = ❓"))
print(generate_result("📀 + 📀 + 📀 = 30 → 📀 = ❓"))


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Incorrect: 🍔 + 🍔 + 🍔 = 15 → 🍔 = ❓
Correct: 🍔 = 5


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Incorrect: 🚜 + 🚜 + 🚜 + 🚜 = 40 → 🚜 = ❓
Correct: To solve this problem, we need to set up an equation based on the given information and then solve for the value of the 🚜. The equation is: 4 * 🚜 = 40. To solve for 🚜, we divide both sides of the equation by 4: 🚜 = 40 / 4. This simplifies to: 🚜 = 10. So the value of the 🚜 is 10. The answer is 10.


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Incorrect: 🐧 + 🐧 = 18 → 🐧 = ❓
Correct: 🐧 = 9


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Incorrect: 🎈 + 🎈 + 🎈 + 🎈 + 🎈 = 25 → 🎈 = ❓
Correct: 🎈 = 5
Incorrect: 📀 + 📀 + 📀 = 30 → 📀 = ❓
Correct: 📀 = 10


In [22]:
import random

def emojiq_brainpower():
    logic = random.randint(40, 100)
    emoji_confusion = 100 - logic
    caffeine = random.randint(10, 50)

    return (
        f"🧠 {logic}% logic, {emoji_confusion}% 'Wait... is 🍕 a number?!' 🤔, "
        f"and {caffeine}% caffeine boost ☕🚀"
    )

print(emojiq_brainpower())


🧠 80% logic, 20% 'Wait... is 🍕 a number?!' 🤔, and 17% caffeine boost ☕🚀
